![alt text](https://www.business.unsw.edu.au/style%20library/asb/assets/images/logo-unsw.png)


# MARK5826: Assignment 1

***Please submit to https://moodle.telt.unsw.edu.au/mod/assign/view.php?id=2364296  with this format Zid_Assignment1.ipynb eg:
z5011973_Assignment1.ipynb. Deadline: Wed 03 Jul @ 10 pm. Penalty -1 a day for a late submission. No cutoff-date. ***

## Question 1 (3 marks):

In user/item -based collaborative filtering, we only print out the movie ids. Now, change the recommended ids to movie names. You will need to process the ml-100k.item file and you can use the following code to read the file.

### Overview of the Movie MetaData

In [4]:
from google.colab import files

# Upload ml-100k.item
files.upload()

{}

In [5]:
import pandas as pd

m_cols = ['movie_id', 'title', 'release_date', 'video_release_date', 'imdb_url']
movie_metadata_df = pd.read_csv('ml-100k.item', sep='|', names=m_cols, usecols=range(5), encoding='latin-1')
movie_metadata_df.head(5)


,movie_id,title,release_date,video_release_date,imdb_url
0,1,Toy Story (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Toy%20Story%2...
1,2,GoldenEye (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?GoldenEye%20(...
2,3,Four Rooms (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Four%20Rooms%...
3,4,Get Shorty (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Get%20Shorty%...
4,5,Copycat (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Copycat%20(1995)


Select 'movie_id' and 'title' from the meta_data dataframe

In [0]:
movie_metadata_df = movie_metadata_df[['movie_id', 'title']]

In [8]:
movie_metadata_df.head(10)

,movie_id,title
0,1,Toy Story (1995)
1,2,GoldenEye (1995)
2,3,Four Rooms (1995)
3,4,Get Shorty (1995)
4,5,Copycat (1995)
5,6,Shanghai Triad (Yao a yao yao dao waipo qiao) ...
6,7,Twelve Monkeys (1995)
7,8,Babe (1995)
8,9,Dead Man Walking (1995)
9,10,Richard III (1995)


### Your Task Starts Here

Now, define a function that takes the recommended item ids and the movie metadata dataframe as arguements, and return a list of titles.

Refer to the Pandas package on how to process the Pandas DataFrame [here](http://pandas.pydata.org/pandas-docs/stable/).

In [0]:
def id2names(recommended_ids, movie_metadata_df):
    # Write your code here
    l = []
    for id in recommended_ids:
      l.append(str(movie_metadata_df[movie_metadata_df.movie_id == id].title.values[0]))
    return l

In [10]:
tmp = movie_metadata_df[movie_metadata_df.movie_id == 10]
tmp.title.values[0]

'Richard III (1995)'

In [11]:
id2names([1,2,10], movie_metadata_df)

['Toy Story (1995)', 'GoldenEye (1995)', 'Richard III (1995)']

In [12]:
id2names([1], movie_metadata_df)

['Toy Story (1995)']

### Try It For Recommendation

Try your code in the one of the recommendation methods you have learned.

For example, user-based collaborative filtering.

#### You do not need to change anything for the following code. But, you need to run it!

In [13]:
from google.colab import files

# Upload MovieLens 100k Dataset
files.upload()

{}

In [0]:
from google.colab import files

# Upload utils.py
files.upload()

{}

In [14]:
# Install the scikit-surprise library
!pip install scikit-surprise

     |████████████████████████████████| 3.3MB 2.8MB/s 
  Stored in directory: /root/.cache/pip/wheels/ec/c0/55/3a28eab06b53c220015063ebbdb81213cd3dcbb72c088251ec
Successfully built scikit-surprise


In [0]:
#import libraries and helper functions
from utils import *
from sklearn.metrics.pairwise import cosine_similarity
import heapq
import pandas as pd
import warnings; warnings.simplefilter('ignore')

In [0]:
def cosSimilarityUser(data):
    # Calculate the Cosine Similarity Matrix
    user_similarity = cosine_similarity(data)
    
    # Preview the Similarity Matrix
    print("Similarity Matrix Sample")
    print(user_similarity[:5, :5])
    print("Similarity Matrix Dimension")
    print(np.shape(user_similarity))
    print("=" * 120)
    
    return user_similarity

In [0]:
def predictUser(ratings, similarity, num_items):
    # The Average Rating Values for Each User
    mean_user_rating = np.repeat(np.array([ratings.mean(axis=1)]), num_items, axis=0).T

    # The Difference Between Each Rating Value and The Average Value
    ratings_diff = ratings - mean_user_rating

    # Calculate the Predicted Score
    pred = mean_user_rating + \
           np.dot(similarity, ratings_diff) / np.array([np.abs(similarity).sum(axis=1)]).T
    
    return pred

In [0]:
def recItemsForOneUser(pred_array, train_array, user, num_rec):
    # Change Training Arrary Into Sparse Matrix
    train_matrix = sp.csr_matrix(train_array)

    # Get the Item IDs in the Training Data For the Specified User
    train_items_for_user = train_matrix.getrow(user).nonzero()[1]

    # Create A Dictionary with Key-Value Pairs as ItemID-PredictedValue Pair
    pred_dict_for_user = dict(zip(np.arange(train_matrix.shape[1]), pred_array[user]))

    # Remove the Key-Value Pairs used in Training
    for iid in train_items_for_user:
        pred_dict_for_user.pop(iid)

    # Select the Top-N Items in The Sorted List
    rec_list_for_user = heapq.nlargest(num_rec, pred_dict_for_user.items(), key=lambda tup: tup[1])

    # Get the Item ID List From the Top-N Tuples
    rec_item_list = [tup[0] for tup in rec_list_for_user]
    return rec_item_list

#### The Main Program (Use your function id2names here)

In [21]:
if __name__ == '__main__':
    # Load Rating Data
    train, test, num_users, num_items, uid_min, iid_min = loadData(test_size=0.2)
    train_array, test_array = train.toarray(), test.toarray()
    
    # Load Movie MetaData
    m_cols = ['movie_id', 'title', 'release_date', 'video_release_date', 'imdb_url']
    movie_metadata_df = pd.read_csv('ml-100k.item', sep='|', names=m_cols, usecols=range(5), encoding='latin-1')
    movie_metadata_df = movie_metadata_df[['movie_id', 'title']]
    
    
    # Similarity And Prediction Matrices (User)
    similarity_user_array = cosSimilarityUser(train_array)
    pred_user_array = predictUser(train_array, similarity_user_array, num_items)

    # Recommendation
    rec_id_list = recItemsForOneUser(pred_user_array, train_array, 257, 10)
    
    # Change the recommended id list into movie title list
    rec_title_list = id2names(rec_id_list, movie_metadata_df)
    
    # Print out the results
    #print("The Recommendation List for User Is: " + rec_title_list)
    print("The Recommendation List for User Is: " + ', '.join(rec_title_list))
    print("=" * 120)

Data Preview:
   uid  iid  ratings       time
0  196  242        3  881250949
1  186  302        3  891717742
2   22  377        1  878887116
3  244   51        2  880606923
4  166  346        1  886397596
Number of Users: 943
Number of Items: 1682
Sample Data: [[5 3 4 ... 0 0 0]]
Similarity Matrix Sample
[[1.         0.16359537 0.03365039 0.07323055 0.30757548]
 [0.16359537 1.         0.08520491 0.16757297 0.07164769]
 [0.03365039 0.08520491 1.         0.19338712 0.02781491]
 [0.07323055 0.16757297 0.19338712 1.         0.03761586]
 [0.30757548 0.07164769 0.02781491 0.03761586 1.        ]]
Similarity Matrix Dimension
(943, 943)
The Recommendation List for User Is: Midnight in the Garden of Good and Evil (1997), Marvin's Room (1996), Secrets & Lies (1996), In & Out (1997), I.Q. (1994), Chasing Amy (1997), Benny & Joon (1993), Kiss the Girls (1997), Apocalypse Now (1979), Smilla's Sense of Snow (1997)


## Question 2 (3 marks): 

Instead of TF-IDF, use other text feature extraction methods to calculate the similarity between two movies. (e.g., token counts)

You can refer to [sklearn's text feature extraction methods](https://scikit-learn.org/stable/modules/classes.html#module-sklearn.feature_extraction.text)

### Including files and folders

In [0]:
from google.colab import files

# Upload the dataset movies_metadata.csv
files.upload()

In [0]:
from google.colab import files

# Upload the dataset links_small.csv
files.upload()


In [0]:
from google.colab import files

# Upload the dataset utils.py
files.upload()

### Cosine Similarity Matrix

In [0]:
def cosSimilarity(matrix):
    similarity_matrix = cosine_similarity(matrix)
    print("Cosine Similarity Matrix Sample:")
    print(similarity_matrix[:5, :5])
    print("=" * 120)
    return similarity_matrix

### Your Task Starts Here (Implement the Word Count Converter)

Refer to [Scikit-Learn](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html#sklearn.feature_extraction.text.CountVectorizer).

In [0]:
from sklearn.feature_extraction.text import CountVectorizer

def wordcount_converter(df):
    # Change the NaN Values into Empty Strings
    df['tagline'] = df['tagline'].fillna('')
    # We only Use the "Overview" and "TagLine" features to Represent Each Movie
    df['description'] = df['overview'] + df['tagline']
    df['description'] = df['description'].fillna('')

    # Word Count Matrix
    # Implement Your Code Here
    vectorizer = CountVectorizer()
    #vectorizer.fit(df['description'])
    count_matrix = vectorizer.fit_transform(df['description'])
    
    print("Count Matrix Shape: " + str(count_matrix.shape))
    print("=" * 120)
    return count_matrix

### Recommendation For One Item

In [0]:
def recForOneItem(movie_list_df, similarity_matrix, title, num_rec):
    # Get the Indices for Each Title
    idx = movie_list_df[title]

    # Get a List of Tuples, Where Each Tuple Contains A Movie Index and The Similarity Score
    similarity_score = list(enumerate(similarity_matrix[idx]))

    # Sort the Above List of Tuples by The Similarity Score in Descendant Order
    similarity_score = heapq.nlargest(num_rec+1, similarity_score, key=lambda tup: tup[1])

    # Get the Titles of the Recommended Movies
    movie_indices = [item[0] for item in similarity_score[1:]]

    return movie_list_df.iloc[movie_indices]

### The Main Program

In [27]:
if __name__ == '__main__':
    # Load The Movie MetaData
    data, movie_list = cb_loadData()
    # Calculate the Word Count Vector of the Each Movie's 'Description' Feature
    matrix = wordcount_converter(data)
    
    # Calculate Cosine Similarity Between Each Movie Pair
    similarity = cosSimilarity(matrix)
    # Generate Recommended Movies
    recommendation = recForOneItem(movie_list, similarity, 'The Godfather', 10)
    
    print("Recommendation List:")
    print(recommendation)

Full Movie MetaData Sample:
   adult                              belongs_to_collection    budget  \
0  False  {'id': 10194, 'name': 'Toy Story Collection', ...  30000000   
1  False                                                NaN  65000000   
2  False  {'id': 119050, 'name': 'Grumpy Old Men Collect...         0   
3  False                                                NaN  16000000   
4  False  {'id': 96871, 'name': 'Father of the Bride Col...         0   

                         genres                              homepage     id  \
0   [Animation, Comedy, Family]  http://toystory.disney.com/toy-story    862   
1  [Adventure, Fantasy, Family]                                   NaN   8844   
2             [Romance, Comedy]                                   NaN  15602   
3      [Comedy, Drama, Romance]                                   NaN  31357   
4                      [Comedy]                                   NaN  11862   

     imdb_id original_language               origina

## Question 3 (3 marks):

Use the FilmTrust Dataset to replace the MovieLens Dataset in the Tutorial.

Choose your own methods (user or item-based collaborative filtering). You can copy the code from the tutorials.

### How to get the dataset ?

Method1:

DataSet DownLoad Link: [here](https://www.librec.net/datasets/filmtrust.zip)

Only use the 'ratings.csv' file

Method2:

Use the dataset from the tutorial folder.

In [0]:
# Your code is here
from google.colab import files
files.upload()

In [25]:
!pip install scikit-surprise

In [26]:
from utils_q3 import *
from surprise.model_selection import cross_validate
from surprise import KNNWithMeans
import warnings; warnings.simplefilter('ignore')

# Load the data
#changed the spr_loadData 
data, n_users, n_items = spr_loadData('ratings.txt')
print("Loading Data Completed!")

#change the way data is loaded 
# Select Algorithm KNNWithMeans and Run It
sim_options = {'name': 'cosine',
               'user_based': True  # compute similarities between users
              }
algo = KNNWithMeans(k=n_users,sim_options=sim_options)
cross_validate(algo, data, measures=['RMSE'], cv=5, verbose=True)

   uid  iid  ratings
0    1    1      2.0
1    1    2      4.0
2    1    3      3.5
3    1    4      3.0
4    1    5      4.0
Loading Data Completed!
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Evaluating RMSE of algorithm KNNWithMeans on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.8351  0.8344  0.8375  0.8422  0.8292  0.8357  0.0042  
Fit time          1.79    1.72    1.64    1.71    1.66    1.71    0.05    
Test time         5.62    5.69    5.64    5.72    5.76    5.68    0.05    


{'fit_time': (1.7911078929901123,
  1.7232348918914795,
  1.644749402999878,
  1.711334228515625,
  1.6552612781524658),
 'test_rmse': array([0.83505447, 0.83440924, 0.83752545, 0.84215059, 0.82916853]),
 'test_time': (5.620956897735596,
  5.686152935028076,
  5.635926961898804,
  5.7156922817230225,
  5.758887052536011)}

In [0]:
#import libraries and helper functions
from utils_q3 import *
from sklearn.metrics.pairwise import cosine_similarity
import heapq
import warnings; warnings.simplefilter('ignore')

In [0]:
def cosSimilarityUser(data):
    # Calculate the Cosine Similarity Matrix
    user_similarity = cosine_similarity(data)
    
    # Preview the Similarity Matrix
    print("Similarity Matrix Sample")
    print(user_similarity[:5, :5])
    print("Similarity Matrix Dimension")
    print(np.shape(user_similarity))
    print("=" * 120)
    
    return user_similarity

In [0]:
def predictUser(ratings, similarity, num_items):
    # The Average Rating Values for Each User
    mean_user_rating = np.repeat(np.array([ratings.mean(axis=1)]), num_items, axis=0).T

    # The Difference Between Each Rating Value and The Average Value
    ratings_diff = ratings - mean_user_rating

    # Calculate the Predicted Score
    pred = mean_user_rating + \
           np.dot(similarity, ratings_diff) / np.array([np.abs(similarity).sum(axis=1)]).T
    
    return pred

In [0]:
def recItemsForOneUser(pred_array, train_array, user, num_rec):
    # Change Training Arrary Into Sparse Matrix
    train_matrix = sp.csr_matrix(train_array)

    # Get the Item IDs in the Training Data For the Specified User
    train_items_for_user = train_matrix.getrow(user).nonzero()[1]

    # Create A Dictionary with Key-Value Pairs as ItemID-PredictedValue Pair
    pred_dict_for_user = dict(zip(np.arange(train_matrix.shape[1]), pred_array[user]))

    # Remove the Key-Value Pairs used in Training
    for iid in train_items_for_user:
        pred_dict_for_user.pop(iid)

    # Select the Top-N Items in The Sorted List
    rec_list_for_user = heapq.nlargest(num_rec, pred_dict_for_user.items(), key=lambda tup: tup[1])

    # Get the Item ID List From the Top-N Tuples
    rec_item_list = [tup[0] for tup in rec_list_for_user]
    return rec_item_list

In [0]:
def Precision_and_Recall(pred_item_list, test_item_list):
    # Calculate the Number of Occurrences of Testing Item IDs in the Prediction Item ID List
    sum_relevant_item = 0
    for item in test_item_list:
        if item in pred_item_list:
            sum_relevant_item += 1

    # Calculate the Precision and Recall Value
    precision = sum_relevant_item / len(pred_item_list)
    recall = sum_relevant_item / len(test_item_list)

    return precision, recall

In [0]:
def calMetrics(train_array, test_array, pred_array, at_K):
    # Get All the User IDs in Test Dataset
    test_matrix = sp.coo_matrix(test_array)
    test_users = test_matrix.row
    test_matrix = test_matrix.tocsr()

    # List to Store the Precision/Recall Value for Each User
    precision_u_at_K = []
    recall_u_at_K = []

    # Loop for Each User
    for u in test_users:
        # Get the Recommendation List for the User in Consideration
        rec_list_u = recItemsForOneUser(pred_array, train_array, u, at_K)

        # Generate an Item ID List For Testing
        item_list_u = test_matrix.getrow(u).nonzero()[1]

        # Calculate the Precision and Recall Value for this User
        precision_u, recall_u = Precision_and_Recall(rec_list_u, item_list_u)

        # Save the Precision/Recall Values
        precision_u_at_K.append(precision_u)
        recall_u_at_K.append(recall_u)

    # Calculate the Average Precision/Recall Values Over All Users
    print("Precision@"+str(at_K)+": "+str(np.mean(precision_u_at_K)))
    print("Recall@"+str(at_K)+": "+str(np.mean(recall_u_at_K)))
    print("=" * 120)

In [34]:
if __name__ == '__main__':
    # Load Data
    train, test, num_users, num_items, uid_min, iid_min = loadData(test_size=0.2)
    train_array, test_array = train.toarray(), test.toarray()
    
    # Similarity And Prediction Matrices (User)
    similarity_user_array = cosSimilarityUser(train_array)
    pred_user_array = predictUser(train_array, similarity_user_array, num_items)

    # Recommendation
    rec_list = recItemsForOneUser(pred_user_array, train_array, 257, 10)
    print("The Recommendation List for User Is: " + str(rec_list+iid_min))
    print("=" * 120)
    
    # Metrics Calculation
    calMetrics(train_array, test_array, pred_user_array, 5)

Data Preview:
   uid  iid  ratings       time
0  196  242        3  881250949
1  186  302        3  891717742
2   22  377        1  878887116
3  244   51        2  880606923
4  166  346        1  886397596
Number of Users: 943
Number of Items: 1682
Sample Data: [[5 3 4 ... 0 0 0]]
Similarity Matrix Sample
[[1.         0.16359537 0.03365039 0.07323055 0.30757548]
 [0.16359537 1.         0.08520491 0.16757297 0.07164769]
 [0.03365039 0.08520491 1.         0.19338712 0.02781491]
 [0.07323055 0.16757297 0.19338712 1.         0.03761586]
 [0.30757548 0.07164769 0.02781491 0.03761586 1.        ]]
Similarity Matrix Dimension
(943, 943)
The Recommendation List for User Is: [313 288 286 302  50 269 748 333 181 245]
Precision@5: 0.49233000000000005
Recall@5: 0.0755


## Question 4 (1 mark):

Create a list of numbers from 1 to 20, then use a ‘while’ loop to print the odd numbers only.

In [0]:
# Your code is here
list = [item for item in range(1,21)]
#print(list)

def odd_num(list):
  l = []
  count = 0
  while(count < 20):
    if count % 2 != 0:
      l.append(count)
    count+=1
  return l

In [36]:
odd_num(list)

[1, 3, 5, 7, 9, 11, 13, 15, 17, 19]

##Question 5 (1 mark):

Using a ‘for’ loop, count the occurrences of the letter ‘a’ in the word ‘Anaconda’ regardless of the case of the letter.

In [0]:
def check_freq(string):
  count = 0
  for char in string:
    if char == 'a' or char == 'A':
      count+=1
  return count

In [38]:
x = 'Anaconda'
check_freq('Anaconda')

3

In [39]:
check_freq('Annie Says hallo ananananananan')

10